In [71]:
from datetime import datetime
import logging
from typing import List
import pandas as pd
import datetime
import os

from camply.containers import AvailableCampsite, SearchWindow
from camply.search import SearchRecreationDotGov, SearchReserveCalifornia
from camply.config import EmailConfig

In [66]:
AreaCSV = pd.read_csv('../rec_area_names.csv')
print(AreaCSV.head())

   RecAreaID                               RecAreaName
0          4                                 WestWorld
1          6                             Lake Pleasant
2          9                      Shadow Mountain Lake
3         10  White Mountains National Recreation Area
4         11                               Folsom Lake


In [67]:
user_df = pd.read_csv('../CampsiteInfo - Sheet1.csv')
print(user_df.head())

  Date of Query         Name                 Rec Area  Weeks in Advance  \
0    11/30/2024  Chad Breece   Yosemite National Park                 3   
1    11/30/2024  Chad Breece  Pinnacles National Park                 3   

   Number of Nights  Weekends Only                Email  
0                 1              1  cmebreece@gmail.com  
1                 1              1  cmebreece@gmail.com  


In [74]:

uniq_users = user_df['Name'].unique()
for user in uniq_users:
    ### NOTE: Need a seperate thread for each user
    #get email address for user
    to_email_address = user_df[user_df['Name'] == user]['Email'].unique()
    #get rec area ids for user
    user_recs = user_df[user_df['Name'] == user]['Rec Area'].unique()
    uniq_recs = [int(AreaCSV[AreaCSV['RecAreaName'] == rec]['RecAreaID'].values[0]) for rec in user_recs]

    #get number of weeks of advanced notice
    adv_notice_weeks = int(user_df[user_df['Name'] == user]['Weeks in Advance'].unique()[0])
    #calculate end date
    adv_notice_days = adv_notice_weeks * 7
    u = datetime.date.today()
    d = datetime.timedelta(days=adv_notice_days)
    start = u
    end = u + d
    print(start, end)
    date_range = SearchWindow(start_date = start, end_date = end)
    weekend_only_bool = bool(user_df[user_df['Name'] == user]['Weekends Only'].unique()[0])
    num_nights = int(user_df[user_df['Name'] == user]['Number of Nights'].unique()[0])
    
    # Search for campsites for user
    camping_finder = SearchRecreationDotGov(search_window=date_range,
                                            recreation_area=uniq_recs,  # Glacier Ntl Park
                                            weekends_only=weekend_only_bool,
                                            nights=num_nights,
                                            equipment=[("Tent", None)],
                                            )
    # Get matching campsites
    matches = camping_finder.get_matching_campsites(log=True, verbose=True,
                            continuous=True,
                            polling_interval=5,
                            search_forever=False,
                            notify_first_try=False,
                            )


2024-12-01 2024-12-22


In [73]:
matches

[AvailableCampsite(campsite_id=53855, booking_date=datetime.datetime(2024, 12, 20, 0, 0), booking_end_date=datetime.datetime(2024, 12, 21, 0, 0), booking_nights=1, campsite_site_name='Group Day Use #GG', campsite_loop_name=None, campsite_type='DailyUse', campsite_occupancy=(0, 1), campsite_use_type='Day Use', availability_status='Available', recreation_area='Angel Island SP', recreation_area_id=614, facility_name='East Group Garrison Picnic Area', facility_id=2149, booking_url='https://www.reservecalifornia.com/Web/Default.aspx#!park/614/2149', location=CampsiteLocation(latitude=37.8604, longitude=-122.4212), permitted_equipment=None, campsite_attributes=None),
 AvailableCampsite(campsite_id=39610, booking_date=datetime.datetime(2024, 12, 20, 0, 0), booking_end_date=datetime.datetime(2024, 12, 21, 0, 0), booking_nights=1, campsite_site_name='Campsite #9', campsite_loop_name=None, campsite_type='Camping', campsite_occupancy=(0, 1), campsite_use_type='Campsite', availability_status='Avai